In [ ]:
# import os
# import random
# import shutil
# from tqdm import tqdm

# # Set random seed for reproducibility
# random.seed(42)

# # Source directory
# data_dir = '/Users/apunnakula/AJ/mlzoomcap-capstone2/stanford-dogs-dataset-copy'

# # Directories for train and validation splits
# train_dir = os.path.join(data_dir, 'train')
# validation_dir = os.path.join(data_dir, 'validation')

# # Create train and validation directories if they do not exist
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(validation_dir, exist_ok=True)

# # Fraction of data to be used for validation
# validation_fraction = 0.2

# # Loop through each breed directory, excluding 'train' and 'validation'
# for breed_dir in tqdm(os.listdir(data_dir)):
#     breed_path = os.path.join(data_dir, breed_dir)

#     # Skip 'train' and 'validation' directories themselves
#     if not os.path.isdir(breed_path) or breed_dir in ['train', 'validation']:
#         continue

#     # List all image files
#     images = os.listdir(breed_path)
#     random.shuffle(images)

#     # Determine split index
#     split_index = int(len(images) * (1 - validation_fraction))

#     # Split images
#     train_images = images[:split_index]
#     validation_images = images[split_index:]

#     # Create directories for the current breed
#     train_breed_dir = os.path.join(train_dir, breed_dir)
#     validation_breed_dir = os.path.join(validation_dir, breed_dir)
#     os.makedirs(train_breed_dir, exist_ok=True)
#     os.makedirs(validation_breed_dir, exist_ok=True)

#     # Move files
#     for img in train_images:
#         shutil.move(os.path.join(breed_path, img), train_breed_dir)

#     for img in validation_images:
#         shutil.move(os.path.join(breed_path, img), validation_breed_dir)

#     # Remove the now-empty breed directory
#     os.rmdir(breed_path)

In [1]:
# data preparation

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

# Set the path to the dataset
train_dir = '/Users/apunnakula/AJ/mlzoomcap-capstone2/stanford-dogs-dataset-copy/train'
validation_dir = '/Users/apunnakula/AJ/mlzoomcap-capstone2/stanford-dogs-dataset-copy/validation'

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

# Normalization for validation
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 16418 images belonging to 120 classes.
Found 4162 images belonging to 120 classes.


In [3]:
# model selection and training

from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

def create_model(model_name):
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    else:
        raise ValueError("Model not recognized")

    # Freeze the base model
    base_model.trainable = False

    # Create a new model on top
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(len(train_generator.class_indices), activation='softmax')
    ])

    return model

# Define a function to compile, train, and evaluate models
def train_evaluate_model(model_name):
    model = create_model(model_name)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    callbacks = [
        ModelCheckpoint(f'{model_name}_best_model.h5', save_best_only=True),
        EarlyStopping(patience=5, restore_best_weights=True)
    ]

    history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,
        callbacks=callbacks
    )

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(validation_generator)
    print(f'{model_name} - Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')
    return val_accuracy

# Train and evaluate each model
model_names = ['VGG16', 'ResNet50', 'MobileNetV2']
val_accuracies = []

for model_name in model_names:
    accuracy = train_evaluate_model(model_name)
    val_accuracies.append(accuracy)

# Find the best model
best_model_index = np.argmax(val_accuracies)
print(f'The best model is {model_names[best_model_index]} with an accuracy of {val_accuracies[best_model_index]}')


Epoch 1/20


2025-01-22 14:30:10.722726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.0245 - loss: 4.7493

/Users/apunnakula/AJ/mlzoomcap-capstone2/capstone2/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


514/514 ━━━━━━━━━━━━━━━━━━━━ 43s 79ms/step - accuracy: 0.0245 - loss: 4.7489 - val_accuracy: 0.0961 - val_loss: 4.1006
Epoch 2/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.1007 - loss: 4.0442

...

514/514 ━━━━━━━━━━━━━━━━━━━━ 43s 84ms/step - accuracy: 0.0169 - loss: 4.7597 - val_accuracy: 0.0221 - val_loss: 4.7104
Epoch 16/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 43s 83ms/step - accuracy: 0.0226 - loss: 4.7382 - val_accuracy: 0.0139 - val_loss: 4.7509
Epoch 17/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 43s 84ms/step - accuracy: 0.0175 - loss: 4.7575 - val_accuracy: 0.0180 - val_loss: 4.7412
Epoch 18/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 42s 81ms/step - accuracy: 0.0168 - loss: 4.7610 - val_accuracy: 0.0173 - val_loss: 4.7124
Epoch 19/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0179 - loss: 4.7448

514/514 ━━━━━━━━━━━━━━━━━━━━ 43s 84ms/step - accuracy: 0.0179 - loss: 4.7448 - val_accuracy: 0.0209 - val_loss: 4.7058
Epoch 20/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 42s 82ms/step - accuracy: 0.0184 - loss: 4.7474 - val_accuracy: 0.0221 - val_loss: 4.7344
131/131 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step - accuracy: 0.0220 - loss: 4.6993
ResNet50 - Validation Loss: 4.705803871154785, Validation Accuracy: 0.020903412252664566


/var/folders/l0/1lksy1zn3hj4fx2pm__f6c2c0000gn/T/ipykernel_30220/3957475068.py:15: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2111 - loss: 4.0108

514/514 ━━━━━━━━━━━━━━━━━━━━ 45s 83ms/step - accuracy: 0.2112 - loss: 4.0096 - val_accuracy: 0.4476 - val_loss: 2.2696
Epoch 2/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 40s 77ms/step - accuracy: 0.3886 - loss: 2.8059 - val_accuracy: 0.4745 - val_loss: 2.4106
Epoch 3/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 41s 79ms/step - accuracy: 0.4146 - loss: 2.8557 - val_accuracy: 0.4784 - val_loss: 2.6519
Epoch 4/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 39s 76ms/step - accuracy: 0.4357 - loss: 2.8838 - val_accuracy: 0.4680 - val_loss: 2.7414
Epoch 5/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 40s 77ms/step - accuracy: 0.4359 - loss: 2.9703 - val_accuracy: 0.4702 - val_loss: 2.8437
Epoch 6/20
514/514 ━━━━━━━━━━━━━━━━━━━━ 40s 79ms/step - accuracy: 0.4195 - loss: 3.3094 - val_accuracy: 0.4702 - val_loss: 3.2388
131/131 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.4604 - loss: 2.2464
MobileNetV2 - Validation Loss: 2.2696259021759033, Validation Accuracy: 0.44762134552001953
The best model is MobileNetV2 with an accuracy of 0.44762134552